# **MIP1_Run1**

In [2]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

ERROR: Operation cancelled by user
ERROR: Operation cancelled by user
ERROR: Operation cancelled by user
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
glpk-utils is already the newest version (5.0-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
  Using cached glpk-0.4.7.tar.gz (161 kB)
  Installing build dependencies ... canceled
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", l

In [1]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [3]:
problem = """You are a city planner, looking to open facilities at some locations. We have a set of customers and a set of possible locations for opening facilities. Each potential location for establishing a facility incurs a fixed annual activation cost, which must be paid if the facility is used, regardless of the service volume it handles. Furthermore, this service volume at each facility is capped at a maximum annual limit. Additionally, there are transportation costs associated with servicing each customer from each facility.
The goal is to minimize the overall costs, which include both the fixed activation costs for any opened facilities and the transportation costs for servicing customers. This must be done while making sure that each customer’s demand is met, each facility does not exceed its maximum service volume, and customers can of course only be serviced by a facility that is opened.
Please formulate this as a mathematical optimization model."""

In [4]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 1
)


In [5]:
print(response1.choices[0].message.content)

Let's denote:

Sets and Indices:

i ∈ I: Set of potential locations for the facilities
j ∈ J: Set of customers

Parameters:

Fi: Fixed annual activation cost of facility at location i
Ti,j: Transportation cost for delivering from facility i to customer j
Qi: Maximum annual service volume at facility i
Dj: Demand of customer j

Variables:

Xi: Binary variable that equals 1 if a facility is opened at location i and 0 otherwise
Yi,j: Amount of service volume delivered from facility i to customer j

Thus, the variables for this optimization problem can be identified as Xi and Yi,j.


In [6]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [7]:
print(response2.choices[0].message.content)

Objective Function:

Minimize ∑_{i ∈ I} (Fi * Xi) + ∑_{i ∈ I} ∑_{j ∈ J} (Ti,j * Yi,j)


In [8]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [9]:
print(response3.choices[0].message.content)

Constraints:

1. A customer's demand must be met:
∑_{i ∈ I} Yi,j >= Dj for all j ∈ J

2. The service volume of a facility must not exceed its maximum volume:
∑_{j ∈ J} Yi,j <= Qi * Xi for all i ∈ I

3. A customer can only be serviced by an opened facility:
Yi,j <= Dj * Xi for all i ∈ I, j ∈ J

4. The service volume delivered to a customer by a facility must be non-negative:
Yi,j >= 0 for all i ∈ I, j ∈ J

5. The decision of whether to open a facility is binary:
Xi ∈ {0, 1} for all i ∈ I


### **Generate Pyomo Code**

In [10]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [11]:
Markdown(response4.choices[0].message.content)

Sure, here's an example of how you might translate this problem to code using `Pyomo`. Note that I'm using random values for parameters `Fi`, `Ti,j`, `Qi`, and `Dj`.

```python
from pyomo.environ import *

# sample data
locs = range(5)  # indices for 5 potential locations
customers = range(10)  # indices for 10 customers

# random parameters
import numpy as np
Fi = np.random.rand(5)  # fixed annual activation cost for each facility
Ti_j = np.random.rand(5, 10)  # transportation cost for each facility-customer pair
Qi = np.random.rand(5) * 100  # maximum annual service volume for each facility
Dj = np.random.rand(10) * 10  # demand of each customer

model = ConcreteModel()

# sets
model.I = Set(initialize=locs)
model.J = Set(initialize=customers)

# parameters
model.F = Param(model.I, initialize={i: Fi[i] for i in locs})
model.T = Param(model.I, model.J, initialize={(i,j): Ti_j[i,j] for i in locs for j in customers})
model.Q = Param(model.I, initialize={i: Qi[i] for i in locs})
model.D = Param(model.J, initialize={j: Dj[j] for j in customers})

# variables
model.X = Var(model.I, within=Binary)
model.Y = Var(model.I, model.J, within=NonNegativeReals)

# objective function
model.obj = Objective(expr=sum(model.F[i]*model.X[i] for i in model.I) + 
                      sum(model.T[i,j]*model.Y[i,j] for i in model.I for j in model.J), sense=minimize)

# constraints
model.demand_constraint = Constraint(model.J, rule=lambda model, j: 
                                      sum(model.Y[i,j] for i in model.I) >= model.D[j])

model.volume_constraint = Constraint(model.I, rule=lambda model, i: 
                                      sum(model.Y[i,j] for j in model.J) <= model.Q[i]*model.X[i])

# solve the model
SolverFactory('glpk').solve(model)
model.pprint()
```

In this code, the function `np.random.rand` is used to generate random float numbers. Make sure to replace this with your actual data.

### **Run the code Generated by GPT4**

In [14]:
from pyomo.environ import *

# sample data
locs = range(5)  # indices for 5 potential locations
customers = range(10)  # indices for 10 customers

# random parameters
import numpy as np
Fi = np.random.rand(5)  # fixed annual activation cost for each facility
Ti_j = np.random.rand(5, 10)  # transportation cost for each facility-customer pair
Qi = np.random.rand(5) * 100  # maximum annual service volume for each facility
Dj = np.random.rand(10) * 10  # demand of each customer

model = ConcreteModel()

# sets
model.I = Set(initialize=locs)
model.J = Set(initialize=customers)

# parameters
model.F = Param(model.I, initialize={i: Fi[i] for i in locs})
model.T = Param(model.I, model.J, initialize={(i,j): Ti_j[i,j] for i in locs for j in customers})
model.Q = Param(model.I, initialize={i: Qi[i] for i in locs})
model.D = Param(model.J, initialize={j: Dj[j] for j in customers})

# variables
model.X = Var(model.I, within=Binary)
model.Y = Var(model.I, model.J, within=NonNegativeReals)

# objective function
model.obj = Objective(expr=sum(model.F[i]*model.X[i] for i in model.I) +
                      sum(model.T[i,j]*model.Y[i,j] for i in model.I for j in model.J), sense=minimize)

# constraints
model.demand_constraint = Constraint(model.J, rule=lambda model, j:
                                      sum(model.Y[i,j] for i in model.I) >= model.D[j])

model.volume_constraint = Constraint(model.I, rule=lambda model, i:
                                      sum(model.Y[i,j] for j in model.J) <= model.Q[i]*model.X[i])

# solve the model
SolverFactory('glpk').solve(model)
model.pprint()

2 Set Declarations
    I : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}
    J : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

4 Param Declarations
    D : Size=10, Index=J, Domain=Any, Default=None, Mutable=False
        Key : Value
          0 :   9.38841226132385
          1 :  6.182069779986615
          2 :   5.47183281070096
          3 :  3.497342356246974
          4 : 0.6873990796549445
          5 :  8.308479986032747
          6 :  3.790253058555132
          7 : 1.7010417870314787
          8 :  3.691051078326476
          9 : 0.9216594229487896
    F : Size=5, Index=I, Domain=Any, Default=None, Mutable=False
        Key : Value
          0 :  0.8082031600779672
          1 : 0.04432465038512279
          2 : 0.03384953406796687
          3 :  0.9271855670724548
          4

In [17]:
from pyomo.environ import *

# sample data
locs = range(1,3)  # indices for 5 potential locations
customers = range(1,6)  # indices for 10 customers

# random parameters
import numpy as np
Fi = {1: 1000, 2: 1000, 3: 1000}  # fixed annual activation cost for each facility
Ti_j = {
    (1, 1): 4,  (1, 2): 6,  (1, 3): 9,
    (2, 1): 5,  (2, 2): 4,  (2, 3): 7,
    (3, 1): 6,  (3, 2): 3,  (3, 3): 4,
    (1, 4): 8,  (2, 4): 5,  (3, 4): 3,
    (1, 5): 10, (2, 5): 8,  (3, 5): 4,
}  # transportation cost for each facility-customer pair
Qi = {1: 500, 2: 500, 3: 500}  # maximum annual service volume for each facility
Dj = {1: 80, 2: 270, 3: 250, 4: 160, 5: 180}  # demand of each customer

model = ConcreteModel()

# sets
model.I = Set(initialize=locs)
model.J = Set(initialize=customers)

# parameters
model.F = Param(model.I, initialize={i: Fi[i] for i in locs})
model.T = Param(model.I, model.J, initialize={(i,j): Ti_j[i,j] for i in locs for j in customers})
model.Q = Param(model.I, initialize={i: Qi[i] for i in locs})
model.D = Param(model.J, initialize={j: Dj[j] for j in customers})

# variables
model.X = Var(model.I, within=Binary)
model.Y = Var(model.I, model.J, within=NonNegativeReals)

# objective function
model.obj = Objective(expr=sum(model.F[i]*model.X[i] for i in model.I) +
                      sum(model.T[i,j]*model.Y[i,j] for i in model.I for j in model.J), sense=minimize)

# constraints
model.demand_constraint = Constraint(model.J, rule=lambda model, j:
                                      sum(model.Y[i,j] for i in model.I) >= model.D[j])

model.volume_constraint = Constraint(model.I, rule=lambda model, i:
                                      sum(model.Y[i,j] for j in model.J) <= model.Q[i]*model.X[i])

# solve the model
SolverFactory('glpk').solve(model)
print(model.obj())

8110.0


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [54]:
from pyomo.environ import *

# sample data
locs = range(1,4)  # indices for 5 potential locations
customers = range(1,6)  # indices for 10 customers

# random parameters
import numpy as np
F = {1: 1000, 2: 1000, 3: 1000}  # fixed annual activation cost for each facility
T = {
    (1, 1): 4,  (2, 1): 6,  (3, 1): 9,
    (1, 2): 5,  (2, 2): 4,  (3, 2): 7,
    (1, 3): 6,  (2, 3): 3,  (3, 3): 4,
    (1,4): 8,  (2,4): 5,  (3,4): 3,
    (1,5): 10, (2,5): 8,  (3,5): 4,
} # transportation cost for each facility-customer pair
Q = {1: 500, 2: 500, 3: 500}  # maximum annual service volume for each facility
D = {1: 80, 2: 270, 3: 250, 4: 160, 5: 180}  # demand of each customer

model = ConcreteModel()

# sets
model.I = Set(initialize=locs)
model.J = Set(initialize=customers)

#Variables
model.X = Var(model.I, within=Binary)
model.Y = Var(model.I, model.J, within=NonNegativeReals)

#Objective
model.obj = Objective(
        expr=sum(F[i] * model.X[i] for i in model.I) +
             sum(T[i, j] * model.Y[i, j] for j in model.J for i in model.I),
        sense=minimize
    )
#Constraints
def demand_rule(model, j):
        return sum(model.Y[i, j] for i in model.I) >= D[j]
model.demand = Constraint(model.J, rule=demand_rule)

def capacity_rule(model, i):
  return sum(model.Y[i, j] for j in model.J) <= Q[i] * model.X[i]
model.capacity = Constraint(model.I, rule=capacity_rule)

def strong_rule(model, i, j):
      return model.Y[i, j] <= D[j] * model.X[i]
model.strong = Constraint(model.I, model.J, rule=strong_rule)

# solve the model
SolverFactory('glpk').solve(model)
print(model.obj())

5610.000000000003
